# Assemble the final Dataframe from tarpmir bindingsites

## Read in bindingsite & Ensembl data

In [1]:
%load_ext autoreload
%autoreload 2
from helper_fcts import *
import sqlite3
import os
import pandas as pd
from pathlib import Path

path = Path('C:/Users/Lena/Documents/auf mega gesyncte Dokumente/Uni/Master/Semester 5/Masterarbeit/alt-splicing-mirna-reg')
path_tarp = Path(path/'tarp-bs')

In [2]:
#executed once, read in from pickled files
ids = read_in_ids(path/'all_mapping_ids.fasta') # mapping of transcript ids to exon ids
chrom_exon_starts, exon_starts, exon_ends = calc_exon_data(path) # exon starts and ends per transcript + exon id
#read in df_gene: gene_id, g_id_v, transcript_id, t_id_v, chromosome, strand, start, end (1-N)
df_gene = pd.read_csv(path/'gene_infos.csv', dtype={'ensembl_gene_id':str,'ensembl_gene_id_version':str,'ensembl_transcript_id':str, 'ensembl_transcript_id_version':str, 'chromosome_name':'category', 'strand':'int8', 'start_position':int, 'end_position':int})
del df_gene['Unnamed: 0']
df_gene.head()

,ensembl_gene_id,ensembl_gene_id_version,ensembl_transcript_id,ensembl_transcript_id_version,chromosome_name,strand,start_position,end_position
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT,1,577,647
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT,1,648,1601
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT,1,1602,1670
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT,1,1671,3229
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT,1,3230,3304


In [106]:
#set bs from feather
bs = pd.read_feather(path/'read_in_bs.feather')

In [9]:
#read in bs
seperate_bs = []
for filename in os.listdir(path_tarp):
    binding_sites = parse_tarp_bs(path_tarp/filename)
    seperate_bs.append(binding_sites)
bs = pd.concat(seperate_bs, axis=0, ignore_index=True) #14 388 + 10 326 = 24 714 rows
bs.to_feather(path/'read_in_bs.feather')
print('All predicted bindingsites were read into pandas.')

All predicted bindingsites were read into pandas.


In [5]:
#parse miRNA and mRNA seed from sequence
miRNA_sequences = parse_seq(path/'input_miRNA.fasta')#2661 miRNAs 
bs['miRNA_seed'] = bs.apply(lambda row: miRNA_sequences[row['miRNA']], axis=1)
mRNA_sequences = parse_seq(path/'cdna6247.fasta') #TODO change to bigger file with all mRNAs or iterate over all files
bs['mRNA_bs_seq'] = bs.apply(lambda row: mRNA_sequences[row['mRNA']][row['bs_start']:row['bs_end']], axis=1)

In [6]:
%load_ext Cython

In [7]:
%%cython
def pos_to_chrom(pos, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if pos >= exon_starts[tid][eid] and pos <= exon_ends[tid][eid]:
                diff = pos - exon_starts[tid][eid]
                chrom_pos = chrom_exon_starts[tid][eid] + diff
                return chrom_pos
        else: print('Error: Either',tid,'or',eid,'not in exon_starts or exon_ends')
    return None

In [10]:
%%cython
def get_eid(start, end, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if (start >= exon_starts[tid][eid]) and (start <= exon_ends[tid][eid]) and (end >= exon_starts[tid][eid]) and (end <= exon_ends[tid][eid]):
                return eid
        else: print('Error: Either',tid,'or',eid,'not in exon_starts or exon_ends')
    return None

In [14]:
#translate genome position relative to transcript to chromosome
#fastest, TODO check for huge files
bs['chrom_bs_start'] = bs.apply(lambda row: pos_to_chrom(row.bs_start, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
bs['chrom_bs_end'] = bs.apply(lambda row: pos_to_chrom(row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
bs['exon_id'] = bs.apply(lambda row: get_eid(row.bs_start, row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
#bs['bs_id'] = bs.index
bs.head() #4.69, 4.9, 4.48s | 13.3s, 13.3s

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,chrom_bs_start,chrom_bs_end,exon_id
0,hsa-let-7a-2-3p,ENST00000576537,1.000000,-25.9,0,0.000156,0.338,0.005183,-0.035634,-11.206441,...,21,16,4,7,1,309,335,1578440,1578466,ENSE00002650258
1,hsa-let-7a-2-3p,ENST00000576537,0.743590,-21.6,1,0.000002,0.338,-0.161796,0.055277,-3.793325,...,20,6,8,3,4,430,450,1578561,1578581,ENSE00002650258
2,hsa-let-7b-5p,ENST00000576537,0.846154,-18.4,1,0.000026,0.441,3.547385,2.945605,-6.324962,...,25,11,5,8,1,105,130,1576707,1578261,None
3,hsa-let-7b-3p,ENST00000576537,0.615385,-20.6,0,0.000143,0.338,-0.021616,-0.047693,-18.306612,...,21,12,9,7,3,315,336,1578446,1578467,ENSE00002650258
4,hsa-let-7b-3p,ENST00000576537,0.538462,-15.3,0,0.000020,0.456,3.773200,3.161275,-9.685015,...,18,8,7,5,1,23,41,1576625,1576643,ENSE00002671000


In [16]:
%%cython
def pos_to_chrom_new(start, end, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        ids_exist = tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]
        both_in_exon = (start >= exon_starts[tid][eid]) and (start <= exon_ends[tid][eid]) and (end >= exon_starts[tid][eid]) and (end <= exon_ends[tid][eid])
        if ids_exist and both_in_exon:
            chrom_start_pos = chrom_exon_starts[tid][eid] + (start - exon_starts[tid][eid])
            chrom_end_pos = chrom_exon_starts[tid][eid] + (end - exon_starts[tid][eid])
            return (eid, chrom_start_pos, chrom_end_pos)
   # if not both_in_exon:
    #    print('Error: The bindingsite spans more than 1 exon with bs length',end-start+1)
    #if not ids_exist:
     #   print('Error: Either',tid,'or',eid,'not in exon_starts or exon_ends')
    return (2,2,2)#None, None, None)

In [17]:
#middle
bs['exon_id'] = np.nan
bs['chrom_bs_start'] = np.nan
bs['chrom_bs_end'] = np.nan
bs[['exon_id','chrom_bs_start','chrom_bs_end']] = bs.apply(lambda row: pos_to_chrom_new(row.bs_start, row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1, result_type='expand')
#bs['bs_id'] = bs.index
bs.head() #6.77s, 6.79, 7.32 | 18.7s

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,exon_id,chrom_bs_start,chrom_bs_end
0,hsa-let-7a-2-3p,ENST00000576537,1.000000,-25.9,0,0.000156,0.338,0.005183,-0.035634,-11.206441,...,21,16,4,7,1,309,335,ENSE00002650258,1578440,1578466
1,hsa-let-7a-2-3p,ENST00000576537,0.743590,-21.6,1,0.000002,0.338,-0.161796,0.055277,-3.793325,...,20,6,8,3,4,430,450,ENSE00002650258,1578561,1578581
2,hsa-let-7b-5p,ENST00000576537,0.846154,-18.4,1,0.000026,0.441,3.547385,2.945605,-6.324962,...,25,11,5,8,1,105,130,2,2,2
3,hsa-let-7b-3p,ENST00000576537,0.615385,-20.6,0,0.000143,0.338,-0.021616,-0.047693,-18.306612,...,21,12,9,7,3,315,336,ENSE00002650258,1578446,1578467
4,hsa-let-7b-3p,ENST00000576537,0.538462,-15.3,0,0.000020,0.456,3.773200,3.161275,-9.685015,...,18,8,7,5,1,23,41,ENSE00002671000,1576625,1576643


In [20]:
bs = pd.read_feather(path/'read_in_bs.feather')#reset

In [19]:
#middle
bs = bs.join(bs.apply(lambda row: pos_to_chrom_new(row.bs_start, row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1, result_type='expand'))
#bs['bs_id'] = bs.index
bs = bs.rename(columns={0: 'exon_id',1:'chrom_bs_start',2:'chrom_bs_end'})
bs.head() #6.60s, 6.67s, 7.59s | 18.3s

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,exon_id,chrom_bs_start,chrom_bs_end
0,hsa-let-7a-2-3p,ENST00000576537,1.000000,-25.9,0,0.000156,0.338,0.005183,-0.035634,-11.206441,...,21,16,4,7,1,309,335,ENSE00002650258,1578440,1578466
1,hsa-let-7a-2-3p,ENST00000576537,0.743590,-21.6,1,0.000002,0.338,-0.161796,0.055277,-3.793325,...,20,6,8,3,4,430,450,ENSE00002650258,1578561,1578581
2,hsa-let-7b-5p,ENST00000576537,0.846154,-18.4,1,0.000026,0.441,3.547385,2.945605,-6.324962,...,25,11,5,8,1,105,130,2,2,2
3,hsa-let-7b-3p,ENST00000576537,0.615385,-20.6,0,0.000143,0.338,-0.021616,-0.047693,-18.306612,...,21,12,9,7,3,315,336,ENSE00002650258,1578446,1578467
4,hsa-let-7b-3p,ENST00000576537,0.538462,-15.3,0,0.000020,0.456,3.773200,3.161275,-9.685015,...,18,8,7,5,1,23,41,ENSE00002671000,1576625,1576643


In [21]:
#slowest
bs['exon_id'] = np.nan
bs['chrom_bs_start'] = np.nan
bs['chrom_bs_end'] = np.nan
bs[['exon_id','chrom_bs_start','chrom_bs_end']] = bs.apply(lambda row: pd.Series(pos_to_chrom_new(row.bs_start, row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts)), axis=1)

#bs['bs_id'] = bs.index
bs.head() #9.49s, 9.59s | 39s

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,exon_id,chrom_bs_start,chrom_bs_end
0,hsa-let-7a-2-3p,ENST00000576537,1.000000,-25.9,0,0.000156,0.338,0.005183,-0.035634,-11.206441,...,21,16,4,7,1,309,335,ENSE00002650258,1578440,1578466
1,hsa-let-7a-2-3p,ENST00000576537,0.743590,-21.6,1,0.000002,0.338,-0.161796,0.055277,-3.793325,...,20,6,8,3,4,430,450,ENSE00002650258,1578561,1578581
2,hsa-let-7b-5p,ENST00000576537,0.846154,-18.4,1,0.000026,0.441,3.547385,2.945605,-6.324962,...,25,11,5,8,1,105,130,2,2,2
3,hsa-let-7b-3p,ENST00000576537,0.615385,-20.6,0,0.000143,0.338,-0.021616,-0.047693,-18.306612,...,21,12,9,7,3,315,336,ENSE00002650258,1578446,1578467
4,hsa-let-7b-3p,ENST00000576537,0.538462,-15.3,0,0.000020,0.456,3.773200,3.161275,-9.685015,...,18,8,7,5,1,23,41,ENSE00002671000,1576625,1576643


## Get TCGA data from Xena

In [30]:
import xenaPython as xena
cohort = 'TCGA PanCanAtlas'
host = xena.PUBLIC_HUBS['pancanAtlasHub']
samples = xena.cohort_samples(host, cohort, None)
samples[0: 10]
dataset = 'EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena'
expression = get_fields(host, dataset, samples, GENES) # list of lists.
expression_by_gene = dict(zip(GENES, expression))      # index by gene.
[expression_by_gene.keys(), GENES[0], expression_by_gene[GENES[0]][0:10]]
#instead of loading everything, only load for every row in bs?

C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\xenaPython\__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


NameError: name 'get_fields' is not defined

In [34]:
hub = "https://toil.xenahubs.net"
dataset = "tcga_RSEM_gene_tpm"

xena.dataset_samples(hub, dataset, None)
xena.dataset_gene_probe_avg(hub, dataset, ['TCGA-61-1910-01'], ['DUSP7'])[0]["scores"][0]

[3.469]

In [33]:
hub = "https://tcga.xenahubs.net"
dataset = "TCGA.KICH.sampleMap/miRNA_HiSeq_gene"
#samples = xena.dataset_samples (hub, dataset, None)
samples = xena.dataset_samples (hub, dataset, 10)
print(samples)
#samples = ["TCGA-02-0047-01","TCGA-02-0055-01","TCGA-02-2483-01"]
probes = ['ENSG00000282740.1', 'ENSG00000000005.5']
[position, [ENSG00000282740_1, ENSG00000000005_5]] = xena.dataset_probe_values (hub, dataset, samples, probes)
ENSG00000282740_1

['TCGA-KN-8426-01', 'TCGA-KN-8419-11', 'TCGA-KL-8332-01', 'TCGA-KN-8423-01', 'TCGA-KO-8405-01', 'TCGA-KL-8337-01', 'TCGA-KN-8424-11', 'TCGA-KN-8422-11', 'TCGA-KL-8334-01', 'TCGA-KO-8404-01']


['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']

In [9]:
%load_ext Cython

In [ ]:
%%cython
def c_split_chrom_column(chrom):
    chrom_list = chrom.split(':')
    chromosome_name = str(chrom_list[0][3:])
    strand = 1 if chrom_list[2] == '+' else -1
    chrom_exon_start = int(chrom_list[1].split('-')[0])
    chrom_exon_end = int(chrom_list[1].split('-')[1])
    return chromosome_name,strand,chrom_exon_start,chrom_exon_end

In [ ]:
def list_to_df(list_, nr_columns, data_type=None):
    data = []
    remove_last_columns = 2 if data_type != None else 0
    nr_rows = int((len(list_)-1)/nr_columns)
    for row in range(nr_rows):
        start = row*nr_columns
        end = (row+1)*nr_columns
        data.append(list_[start:end-remove_last_columns])
    df = pd.DataFrame(data[1:], columns = data[0]) #use first line as header
    if data_type=='miRNA':
        df = df.astype({'miRNA_ID':str, 'read_count':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
    elif data_type=='exon':
        df = df.astype({'exon':str, 'raw_counts':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
        #TODO change this to seperate lines cause far faster (see calc_chrom_pos)
        df[['chromosome_name','strand','chrom_exon_start','chrom_exon_end']] = df.exon.apply(lambda x: pd.Series(c_split_chrom_column(str(x))))
        df[['raw_counts','chrom_exon_start','chrom_exon_end']] = df[['raw_counts','chrom_exon_start','chrom_exon_end']].apply(pd.to_numeric,downcast='unsigned')
        df[['chromosome_name','strand']] = df[['chromosome_name','strand']].astype('str').astype('category')
        del df['exon']
    return df

In [ ]:
#UCSC Xena API, get list of files which contain miRNA expression per case_id there

primary_site='Kidney'
download_files_from_legacy('miRNA',3000,primary_site,write=True) #2029 #70min
#download_files_from_legacy('exon',2000,primary_site,write=True) #1588 #35h

In [6]:
#read in downloaded counts from feather files
#TODO remove later when all data there
n_all_mirna_dfs = {}
dir_ = Path(path/'mirna')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_mirna_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
    
n_all_exon_dfs = {}
dir_ = Path(path/'exon')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_exon_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
#mirna_counts[['mirna_read_count']].astype('int') #TODO do this before saving while getting it from API
#how to get from API? per caseid?
print(len(n_all_exon_dfs),len(n_all_mirna_dfs)) #yes this is right, read in from path

1588 2013


In [12]:
#get subset counts of bs and merge with df_gene
counts = bs[['bs_id', 'miRNA', 'mRNA', 'chrom_bs_start', 'chrom_bs_end']].copy()
counts = counts.merge(df_gene[['ensembl_transcript_id','chromosome_name','strand']], left_on='mRNA', right_on='ensembl_transcript_id', how='left')
counts[['chromosome_name','strand','miRNA','mRNA']] = counts[['chromosome_name','strand','miRNA','mRNA']].astype('str').astype('category')
del counts['ensembl_transcript_id']
counts.head()

,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end
0,0,hsa-let-7a-5p,ENST00000596924,58346872,58346895
1,1,hsa-let-7a-5p,ENST00000596924,58346310,58346332
2,2,hsa-let-7a-5p,ENST00000596924,58345845,58345870
3,3,hsa-let-7a-2-3p,ENST00000596924,58345558,58345576
4,4,hsa-let-7a-2-3p,ENST00000596924,58345880,58345898


In [ ]:
#mapping miRNA ID to mature ID
mirna_counts = all_mirna_dfs[case_id]

trans = pd.DataFrame.from_dict(read_in_miRNA(path/'miRNA2mature.fasta'), orient='index')#,columns=["mirna", "mature"])
trans = trans.rename(columns={0: "miRNA_ID"})
trans[['miRNA_ID']] = trans[['miRNA_ID']].astype('str').astype('category')

#mapping miRNA to miRNA family
mirna_families = read_in_mirna_families(path/'mapping_miRNA_families.fasta')
trans = trans.join(mirna_families,how='inner')

In [ ]:
#for filename in os.listdir(caseids):

## Execute this for every caseid

### Map miRNA to counts

In [29]:
counts = counts.join(trans,on='miRNA').merge(mirna_counts[['miRNA_ID','read_count']],on='miRNA_ID')
#bs_mirnas.drop(['miRNA','miRNA_ID'], axis=1, inplace=True)
counts.head()

#2693 with hsa in miRNA2mature / 49156 overall in miRNA2mature (also other species eg cel)
#2656 if inner join cause only 2656 families
#TODO check if miRNA counts from TCGA are more different miRNAs?
#TODO is inner right? are some in important counts None?

KeyError: '24ea3cd5-c2e6-44f7-9658-78af7befeef7'

In [227]:
bs_mirnas[bs_mirnas.exon_id == 216595]

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
7,216595,0,hsa-let-7b-5p,hsa-let-7b,let-7,55647
14,216595,0,hsa-let-7d-5p,hsa-let-7d,let-7,3767
25,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
33,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
46,216595,0,hsa-miR-15a-3p,hsa-mir-15a,mir-15,2258
...,...,...,...,...,...,...
12562,216595,0,hsa-miR-5692a,hsa-mir-5692a-2,mir-5692,0
12563,216595,0,hsa-miR-548az-5p,hsa-mir-548az,mir-548,0
12566,216595,0,hsa-miR-8062,hsa-mir-8062,hsa-miR-8062,0
12571,216595,0,hsa-miR-8064,hsa-mir-8064,hsa-miR-8064,0


In [226]:
bs_mirnas[bs_mirnas.mirna_family =='hsa-miR-1229-5p']

# 1 right? count is added for exonid 216595, but do i already add it before ?
# 2 also : exon_id + mirna double (3368 & 3370) wrong!!

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
3365,216603,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3368,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3370,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3371,216596,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4


### Map exons to counts

In [ ]:
# Get exonid for exon instead of introducing exon_id
#TODO this is not necessary anymore if the new exon counts work!!!!
#instead try directly with ensembl start & end match - if it doesnt work 
#-> try download from https://www.gencodegenes.org/human/ (GRCh38.p13) if same reference genome
bs['exon_id'] = np.nan 

In [23]:
bs.to_sql('bs', con=conn)
conn = sqlite3.connect('db.db')
c = conn.cursor()

In [27]:
#print sqlite3 table 
qry = '''
    select * from bs_tap
    '''
c.execute(qry)
conn.commit()
c.fetchall()
pd.read_sql_query(qry,conn)

,index,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,number_of_pairings,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end
0,0,hsa-let-7a-5p,ENST00000596924,1.000000,-22.3,1,1.489720e-02,0.441,0.000000,0.000000,-4.849714,14,16,14,1,2,5,1695,1718
1,1,hsa-let-7a-5p,ENST00000596924,0.923077,-20.9,1,1.141923e-01,0.471,0.000000,0.000000,-5.850804,19,22,12,1,7,1,1133,1155
2,2,hsa-let-7a-5p,ENST00000596924,0.846154,-20.1,1,8.216239e-03,0.382,0.000000,0.000000,-4.993964,19,25,8,1,6,2,668,693
3,3,hsa-let-7a-2-3p,ENST00000596924,1.000000,-23.2,0,9.643462e-07,0.382,0.000000,0.000000,-13.665600,14,18,11,1,5,2,381,399
4,4,hsa-let-7a-2-3p,ENST00000596924,1.000000,-21.4,0,7.641573e-06,0.456,0.000000,0.000000,-8.968752,15,18,9,9,4,2,703,721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24709,24709,hsa-miR-12131,ENST00000263100,1.000000,-25.1,1,2.668984e-04,0.382,0.333260,0.191636,-5.939295,17,21,11,1,6,1,555,576
24710,24710,hsa-miR-12131,ENST00000263100,0.743590,-20.6,1,1.509081e-02,0.250,-0.148416,0.599113,-4.170624,15,17,8,1,3,5,1090,1107
24711,24711,hsa-miR-12131,ENST00000263100,0.884615,-20.7,1,1.197306e-05,0.353,0.143539,0.380902,-5.529864,18,23,8,1,6,2,615,638
24712,24712,hsa-miR-12133,ENST00000263100,0.717949,-20.2,1,1.061500e-06,0.353,-0.147900,0.224627,-4.906073,19,30,8,1,5,3,1561,1591


In [28]:
#DELETE table from sqlite3 DB
c.execute("DROP TABLE bs_tap;")
conn.commit()

In [19]:
exon_counts = n_all_exon_dfs
exon_counts.head()

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts
index,,,,,
0,1,11874,12227,1,15.0
1,1,12595,12721,1,3.0
2,1,12613,12721,1,3.0
3,1,12646,12697,1,2.0
4,1,13221,14409,1,67.0


In [ ]:
counts = counts.rename(columns={'read_count': "mirna_read_count",'raw_counts':'exon_raw_counts','miRNA family name':'mirna_family'})

In [28]:
#for a large data set, you will likely get a significant speed increase by creating an index for any column(s) used in the join condition. 
#https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#instead use SQL
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"raw_counts": 'INTEGER', "exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs inner join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
#only return bs with minimum 1 exon_id cause inner join (else left join)
#automatically filters out all binding sites that have more than 1 exon_id, if u sont want that use outer join -> actually wrong, it uses the first instead of leaving it out!

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts,exon_id,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end
0,19,58352928,58353197,-1,0,216603,2,hsa-let-7b-5p,ENST00000600966,58352987,58353006
1,19,58351391,58351687,-1,0,216600,3,hsa-let-7b-5p,ENST00000600966,58351458,58351475
2,19,58352283,58352555,-1,0,216602,4,hsa-let-7b-5p,ENST00000600966,58352400,58352420
3,19,58351391,58351687,-1,0,216600,5,hsa-let-7b-3p,ENST00000600966,58351648,58351684
4,19,58351391,58351687,-1,0,216600,8,hsa-let-7d-5p,ENST00000600966,58351399,58351420


In [158]:
#visualize amount exons by bindingsite | amount bindingsites by exon
import sqlite3
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
important_counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
new_exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs left join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
bs_exons2 = pd.read_sql_query(qry, conn)
bs_exons2.head()
#bs_exons2.groupby('bs_id').count().raw_counts.hist()
#bs_exons2.groupby('exon_id').count()

## Put all counts into one & pivot table

In [ ]:
#TODO put all different caseids sqlite tables together into one
#INSERT INTO artists_backup 
#SELECT ArtistId, Name
#FROM artists;

In [ ]:
#output: for each case_id for each bs_id: interesting miRNA expression + exon expression
#unstack family counts from 2 columns to several columns (mirna_family=name,mirna_read_count=value)
pivoted = bs_mirnas.copy()
pivoted = pivoted.pivot_table(values='mirna_read_count', index=['exon_id', 'exon_raw_counts'], columns='mirna_family', aggfunc='sum', fill_value=0)
pivoted#.head()
#TODO debug does this do it right OR use sqllite3

In [ ]:
#TODO pivot in sqlite3 
#https://modern-sql.com/use-case/pivot
#https://stackoverflow.com/questions/1237068/how-to-pivot-in-sqlite-or-i-e-select-in-wide-format-a-table-stored-in-long-form

In [ ]:
#visualize amount exons by bindingsite
#bs_exons.groupby('bs_id').count().raw_counts.hist()

In [274]:
#TODO check - is nr mirna downloaded also here? if not: mistake
print(X.shape,y[0].shape)

(8, 1680) (8,)


# Elastic Net Regression

In [ ]:
import sys
sys.getsizeof(pivoted)#TODO

In [276]:
#regression btw miRNA expression, exon expression
import sklearn
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
#faster if np.array(x, order='F')
#If you are interested in controlling the L1 and L2 penalty separately, keep in mind that this is equivalent to:
#a * L1 + b * L2 where: alpha = a + b and l1_ratio = a / (a + b)
X = np.array(pivoted)
y = np.array(pivoted.index.get_level_values('exon_raw_counts'))

regr = ElasticNet(random_state=0)
regr.fit(X, y)
print(regr.coef_)
print(regr.intercept_)

[0. 0. 0. ... 0. 0. 0.]
0.0


C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(
